# Activity 2 - Data Visualization
**2020 Data Labs REU**

*Written by Sage Lichtenwalner, Rutgers University, June 6, 2020*

In this notebook we will cover some of basics of plotting in python, primarily using the [matplotlib](https://matplotlib.org) library.  We've actually already used this library, as it is built into the pandas and xarray libraries to provide quick plotting capabilities.  But if we want to customize our charts, it's often better to create them directly using matplotlib function calls.

The examples today will continue to use the mooring timeseries data available from [NDBC](https://www.ndbc.noaa.gov), but we will also look at profiles from the [ARGO](https://www.aoml.noaa.gov/phod/argo/) drifter network, which will allow use to create some other graph types you commonly see in oceanography, including profiles and TS diagrams.

In [0]:
# Notebook setup
import xarray as xr
!pip install netcdf4

import matplotlib.pyplot as plt

# NDBC Timeseries
Following our example from yesterday, let's load some timeseries data from an NDBC mooring.  We will use this dataset to show how to customize your plot.

In [0]:
# Open dataset
ds = xr.open_dataset('https://dods.ndbc.noaa.gov/thredds/dodsC/data/stdmet/44025/44025.ncml')

# Subset the dataset to 1 year
ds = ds.sel(time=slice('2019-01-01','2020-01-01'))

## Convert Xarray Dataset to Pandas Dataframe
Yesterday we used the power of Xarray to load our NDBC dataset directly from a Thredds server.  Xarray is great, especially when dealing with 3D or 4D datasets. But it can overcomplicate things.  For example, our NDBC dataset actually loads with 3 dimensions (time, latitude and longitude), but we only need 1 (time).

Here are a few example plotting calls.  Can you tell what's different in the output for each?

In [0]:
# Built in xarray plotting
# ds.sea_surface_temperature.plot();

# Plot using matplotlib - This won't work
# plt.plot(ds.sea_surface_temperature);

# Plot using matplotlib - This will, but the units are wrong
# plt.plot(ds.sea_surface_temperature.squeeze())

# Plot using matplotlib - Correctly plotted with time
# plt.plot(ds.time,ds.sea_surface_temperature.squeeze())

To simply things, we can convert our Xarray Dataset to a Pandas Dataframe, which will give use something like a spreadsheet of columns for each variable, and rows for each measurement time.

Here's how easy it is to convert.

In [0]:
# Convert to Pandas Dataframe
df = ds.to_dataframe()
df.head()

Unfortunately, there's still a bit of complexity here because of the multi-dimensional index.  If we try to plot this now, we get some crazy labels.

In [0]:
df.sea_surface_temperature.plot()

Here's how we can properly convert this Dataset to a Dataframe.

In [0]:
# Convert to Pandas Dataframe
df = ds.to_dataframe().reset_index().set_index('time')
df.head()

In [0]:
# Yes, even Pandas has built in plotting
df.sea_surface_temperature.plot()

And now we're off to the races.

## Customizing Timeseries Plots
Matplotlib provides quite a few ways to customize your plot.  

### Customizing Lines
Here are some of the more common parameters you will typically use when creating your plot.

* linewidth - For example 0.5, 1, 2...
* linestyle - For example '-','--', or ':' or other [basic](https://matplotlib.org/api/_as_gen/matplotlib.lines.Line2D.html#matplotlib.lines.Line2D.set_linestyle) or [advanced](https://matplotlib.org/gallery/lines_bars_and_markers/linestyles.html) styles
* [color](https://matplotlib.org/gallery/color/named_colors.html)
* [marker](https://matplotlib.org/api/markers_api.html#module-matplotlib.markers)
* label - The name of the line, used in a legend

For reference and inspiration, you can also check out the [Matplotlib Gallery](https://matplotlib.org/gallery/index.html).

In [0]:
# Line Example
plt.plot(df.index,df.sea_surface_temperature, color='red', linewidth=3)

In [0]:
# Custom Markers Example
plt.plot(df.index,df.sea_surface_temperature, color='red', linestyle='', 
         marker='d', markerfacecolor='b', markeredgecolor='g', markersize=5)

In [0]:
# Your Turn - Create a graph of air temperature using blue dots

### Customizing the Axis
* Axis Title: `plt.title('Title')`
* Legend: `plt.legend()`
* Axes Labels: `plt.xlabel('Time')` or `plt.ylabel('Temperature')`
* Axes Limits: `plt.ylim([-5,5])`


In [0]:
# Incomplete Example
plt.plot(df.index,df.air_temperature, color='red')
plt.plot(df.index,df.sea_surface_temperature, color='blue', label='Sea Surface Temp')
plt.legend();

In [0]:
# Your turn - Fix the legend, and add a title and y label to the above plot

### Adding Subplots
We can also create a figure with multiple plots using the subplot feature.

In [0]:
# Subplot example
fig, (ax1,ax2) = plt.subplots(2,1, sharex=True, figsize=(10,6))

df.air_temperature.plot(ax=ax1)
df.sea_surface_temperature.plot(ax=ax1)
df.wind_spd.plot(ax=ax2, marker='.',linestyle='',markersize=1)

ax1.legend()
ax1.set_ylabel('Temperature (C)')
ax2.set_ylabel('Wind Speed (m/s)')
ax1.set_title('NDBC Station 44025');

In [0]:
# Your Turn - Recreate the above plot with a 3rd subplot using another variable

## Scatterplot
When two variables are plotted against each other, this is typically called a scatterplot.  They are really no different than the plots we crated above.  We just need to pick two variables, and use a marker instead of a line.

In [0]:
plt.plot(df.sea_surface_temperature,df.air_temperature, linestyle='', marker='.', markersize=3)
# plt.scatter(df.sea_surface_temperature, df.air_temperature, s=3) #Another way
plt.xlabel('Sea Surface Temperature (C)')
plt.ylabel('Air Temperature (C)')
plt.title('NDBC Station 44025 from 1/1/2019 to 1/1/2020');

In [0]:
# Your Turn - Create a scatterplot of winds vs. waves

## Histogram
We can easily create [histograms](https://matplotlib.org/gallery/statistics/hist.html?highlight=histogram) of a single variable.  Use the `bins` parameter to increase or decrease the number of data bins.

In [0]:
# We can also easily create histograms
df['sea_surface_temperature'].hist(bins=50);

## Box Plots
You can also create box plots of your data rather easily.  You could also create [violin plots](https://matplotlib.org/3.2.1/gallery/statistics/boxplot_vs_violin.html).

In [0]:
# And boxplots
df[['sea_surface_temperature','air_temperature','wind_spd']].plot.box(vert=False);

# ARGO Drifter Data
While timeseries datasets at fixed points are very useful, the ocean is a deep place with a lot of interesting features and processes that vary with depth.  

Thus, it's also important to be able to visualize data in ways that emphasize the depth dimension, just as we've emphasized the time dimension above.

To do so, we're going to load data from the [Global Argo Float](http://www.argo.ucsd.edu) program, which had 3,962 active drifters around the world as of 6/6/2020.
![Argo Floats](http://www.argo.ucsd.edu/status.jpg)

Unfortunately, because of the complexity and international nature of the program, there isn't one "perfect" source to retrieve Argo data, or even to search for drifters you may be interested in.  However, I found the following sites helpful.

* [SCCOM Floats](https://www.mbari.org/science/upper-ocean-systems/chemical-sensor-group/soccom-float-visualization/) - A great interactive visualization of active floats in the Southern Ocean. You can also create quick plots and download CSV formatted data using [SOCCOMViz](https://www.mbari.org/science/upper-ocean-systems/chemical-sensor-group/soccomviz/).
* [Global Argo Data Repository](https://www.ncei.noaa.gov/products/global-argo-data-repository) - A great place to grab drifter data files in netcdf format, once you know what drifter ID you want. (We'll actually use this site below.)
* [Euro Argo Map](https://fleetmonitoring.euro-argo.eu/dashboard) - A relatively new interactive portal to search for drifters and see their recent tracks and data.
* [A beginner's guide to accessing Argo data](http://www.argo.ucsd.edu/Argo_date_guide.html) - More information on the data.
* [Old Euro Argo Site](https://www.euro-argo.eu/Activities/Data-Management/Access-to-data-note-this-website-is-currently-under-revision-and-will-be-updated-in-2020) - May or may not work 

For the purposes of this activity, I picked an active drifter at random. Let's see what #5906017 can tell us about the ocean.

In [0]:
# Let's download a datafile to our serve (see the files tab)
!wget https://data.nodc.noaa.gov/argo/gadr/data/aoml/5906017/nodc_5906017_prof.nc

In [0]:
# Load the dataset using xarray
data = xr.open_dataset('nodc_5906017_prof.nc')
data

As you can see, this dataset is a bit more complicated than our mooring timeseries dataset.  

Argo uses the dimensions n_levels (for depth) and n_prof (for time).

In [0]:
# Quick Timeseries Profile plot of Temperature
data.temp_adjusted.T.plot()
plt.gca().invert_yaxis()

*Note, when possible, you should use the `_adjusted` variables, as those have been corrected.  However, if you are interested in more recent data that hasn't been corrected yet, you will need to use the regular variable names (e.g. temp instead of temp_adjusted).*

## Profile Plot
Now that we have the dataset loaded, let's create a profile plot of a single profile.

In [0]:
# Profile Plot
nprof = 25 #Specify a profile to plot
plt.plot(data.temp_adjusted[nprof], data.pres_adjusted[nprof])

plt.xlabel('Temperature (C)')
plt.ylabel('Pressure (dbar)')
plt.title('Argo Profile from %s' % data.juld[nprof].dt.strftime('%a, %b %d %H:%M').values)

plt.gca().invert_yaxis() #Flip the y-axis

Now let's get a little fancier and plot profiles of 2 variables.

In [0]:
# Profile Plot
# Subplot example
fig, (ax1,ax2) = plt.subplots(1,2, sharey=True, figsize=(10,6))

nprof = 0 # Fist profile
ax1.plot(data.temp_adjusted[nprof], data.pres_adjusted[nprof], label=data.juld[nprof].dt.strftime('%Y-%m-%d').values)
ax2.plot(data.psal_adjusted[nprof], data.pres_adjusted[nprof])

nprof = 25 # Middle-ish profile
ax1.plot(data.temp_adjusted[nprof], data.pres_adjusted[nprof], label=data.juld[nprof].dt.strftime('%Y-%m-%d').values)
ax2.plot(data.psal_adjusted[nprof], data.pres_adjusted[nprof])

nprof = -1 # Last profile
ax1.plot(data.temp_adjusted[nprof], data.pres_adjusted[nprof], label=data.juld[nprof].dt.strftime('%Y-%m-%d').values)
ax2.plot(data.psal_adjusted[nprof], data.pres_adjusted[nprof])

ax1.set_ylabel('Pressure (dbar)')
ax1.set_xlabel('Temperature (C)')
ax2.set_xlabel('Salinity')
ax1.invert_yaxis()
ax1.legend()

# Add some gridlines
ax1.grid()
ax2.grid()

# Add a super title
fig.suptitle('Argo Float #%d' % data.platform_number[nprof].values, fontweight='bold', fontsize=16);

### Activity Break
Repeat the steps above to create profile plot of temperature and salinity using Float 5905077.  

Note that this float has a lot of other variables that could be investigated.

In [0]:
# Your Turn - Download the data

In [0]:
# Your Turn - Load the data

In [0]:
# Your Turn - Profile plot

## T-S Diagram
Another popular plot in oceanography is the [T-S Diagram](https://en.wikipedia.org/wiki/Temperature–salinity_diagram), or Temperature-Salinity plot.  It is commonly used to diagnose water masses in the global ocean, and to compare density stability of a water column profile (especially when lines of constant density are also included).

Basically, it's just a scatterplot of temp and salinity.

In [0]:
# TS Diagram
nprof = 25 #Selected profile
plt.scatter(data.psal_adjusted[nprof], data.temp_adjusted[nprof])
plt.xlabel('Salinity')
plt.ylabel('Temperature (°C)')
plt.grid()

plt.title('Argo Float #%d' % data.platform_number[nprof].values, fontweight='bold');

That's nice, but we can also use a colored scatterplot to show the depth dimension.

In [0]:
# T-S Diagram with depth
plt.figure(figsize=(8,6))

nprof = 25 #Selected profile
plt.scatter(data.psal_adjusted[nprof], data.temp_adjusted[nprof], c=data.pres_adjusted[nprof], cmap='viridis_r')
plt.xlabel('Salinity');
plt.ylabel('Temperature (°C)')

cbh = plt.colorbar();
cbh.set_label('Pressure (dbar)')

We used the default python colorbar for this plot (viridis), but there is a much larger [colormap collection](https://matplotlib.org/users/colormaps.html) available.

With a little bit more code, we can add lines of constant density.  This was adapted from the [Ocean Python T-S Diagram](https://oceanpython.org/2013/02/17/t-s-diagram/) example, but we will use meshgrid instead, since it makes the code a bit simpler.


To calculate density, we will need the wonderful [seawater](https://pythonhosted.org/seawater/index.html) library.

In [0]:
!pip install seawater
import seawater
import numpy as np

In [0]:
# TS Diagram with density contours
plt.figure(figsize=(8,6))

# Calculate the density lines
x = np.arange(33, 35, .1)
y = np.arange(2, 23, .5)
X, Y = np.meshgrid(x, y)
Z = seawater.eos80.dens0(X,Y) - 1000 # Substract 1000 to convert to sigma-t

# Plot the contour lines
CS = plt.contour(X, Y, Z, colors='grey', linestyles='dashed', levels=np.arange(22,30,.5))
plt.clabel(CS, inline=1, fontsize=10, fmt='%0.1f')

# Plot the data
nprof = 25 #Selected profile
plt.scatter(data.psal_adjusted[nprof], data.temp_adjusted[nprof], c=data.pres_adjusted[nprof], cmap='viridis_r')
plt.xlabel('Salinity');
plt.ylabel('Temperature (°C)')
plt.title('Argo Float #%d on %s' % (data.platform_number[nprof].values, data.juld[nprof].dt.strftime('%Y-%m-%d').values), fontweight='bold');

# Add a colorbar
cbh = plt.colorbar(label='Pressure (dbar)');

## Float Track Map

In [0]:
# Simple map of a float track
plt.figure(figsize=(8,6))
plt.plot(data.longitude, data.latitude, c='lightgrey')
plt.scatter(data.longitude, data.latitude, c=data.juld, cmap='RdYlBu')

# Crude profile labels
for jj in [1,25,-1]:
  plt.text(data.longitude[jj]+.02, data.latitude[jj]+.02, data.n_prof[jj].values)

# Add a colorbar
cbar = plt.colorbar();

# Fix the colorbar ticks
import pandas as pd # We need pandas for this
cbar.ax.set_yticklabels(pd.to_datetime(cbar.get_ticks()).strftime(date_format='%Y-%m-%d'));

# Set the aspect ratio to pseudo-Mercator
plt.gca().set_aspect(1 / np.cos(np.deg2rad( np.mean(plt.ylim()) )))

## Fancier Maps with Cartopy
The cartopy library provides a lot of great features to create your own custom maps, though it takes a lot of patience to get it to work correctly.

In [0]:
# Install Cartopy in Google Colab
!apt-get -V -y -qq install python-cartopy python3-cartopy
!pip uninstall shapely -y
!pip install shapely --no-binary shapely

In [0]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
from cartopy.feature import NaturalEarthFeature

In [0]:
fig = plt.figure(figsize=(6.4,4.8),dpi=150)

ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_extent([-130, -105, 22, 45])

# Basemaps
states = NaturalEarthFeature(category="cultural", scale="10m",
                             facecolor="none",
                             name="admin_1_states_provinces_shp")
ax.add_feature(states, linewidth=.5, edgecolor="black",facecolor='grey')
ax.coastlines('50m', linewidth=1)

ax.plot(data.longitude, data.latitude, linewidth=3, color='royalblue');


For another example, check out this [notebook](https://github.com/ooi-data-lab/data-lab-workshops/blob/master/March2019/Activities/DL_March_Anoxia_v4.ipynb) which plots a glider track, colored by date, along with lat/lon labels and station locations.

# Additional Resources
If you are interested in seeing some additional examples of the plotting features available in python, I encourage you to visit the following pages.

* [Matplotlib Examples](https://matplotlib.org/gallery/index.html) - See what else this library can do.
* [Seaborn Gallery](https://seaborn.pydata.org/examples/index.html) - A great library for creating good-looking common statistical graphs.
* [Altair Example Gallery](https://altair-viz.github.io/gallery/index.html) - A more advanced tool for creating interactive graphs.
* [Python Graph Gallery](https://python-graph-gallery.com/) - A great resource for learning about common data visualization styles and how to create them in python.